<a href="https://colab.research.google.com/github/mehrinshamim/Practice-Git/blob/main/01_simple_qa_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# LangChain Q&A Bot - Learning Tutorial
# Professor's Guide: From Zero to Working Bot

## 🎓 LEARNING OBJECTIVES:
- Understand LangChain's core components
- Build a conversational Q&A bot step by step  
- Learn prompt engineering fundamentals
- Implement conversation memory
- Master the chain-of-thought approach

## 📚 What we'll cover:
1. LangChain basics & setup
2. Prompts & Templates  
3. LLMs & Integration
4. Output Parsing
5. Memory Systems
6. Putting it all together
"""



In [1]:
# ================================
# STEP 1: INSTALLATION & SETUP
# ================================
print("🔧 Setting up our learning environment...")

# Install required packages
!pip install langchain langchain-groq python-dotenv

# Import libraries we'll use
import os
from getpass import getpass
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema import BaseOutputParser
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.schema import HumanMessage, AIMessage

print("✅ Setup complete!")

🔧 Setting up our learning environment...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 11.0 MB/s eta 0:00:00
✅ Setup complete!


In [2]:
# ================================
# STEP 2: API KEY SETUP
# ================================
print("\n🔑 API Key Setup")
print("Go to https://console.groq.com/keys to get your free API key")

# Secure way to input API key in Colab
groq_api_key = getpass("Enter your Groq API key: ")
os.environ["GROQ_API_KEY"] = groq_api_key

print("✅ API key configured!")


🔑 API Key Setup
Go to https://console.groq.com/keys to get your free API key
Enter your Groq API key: ··········
✅ API key configured!


In [3]:
# ================================
# LESSON 1: UNDERSTANDING LLMS
# ================================
print("\n" + "="*50)
print("📖 LESSON 1: Your First LLM Call")
print("="*50)

# Initialize the LLM - this is your "brain"
llm = ChatGroq(
    model="llama3-8b-8192",  # Fast, good for learning
    temperature=0.7,         # Controls randomness (0=deterministic, 1=creative)
    max_tokens=150          # Limit response length
)

print("🧠 LLM initialized! Let's test it...")

# Direct LLM call - the simplest way
response = llm.invoke("What are the key ingredients in a classic carbonara?")
print("🤖 LLM Response:")
print(response.content)

print("\n💡 KEY CONCEPT: This is a 'stateless' call - the LLM has no memory!")


📖 LESSON 1: Your First LLM Call
🧠 LLM initialized! Let's test it...
🤖 LLM Response:
A classic carbonara! A delicious and rich Italian pasta dish that's surprisingly simple to make. The key ingredients in a traditional carbonara are:

1. **Spaghetti**: Long, thin strands of pasta made from durum wheat semolina.
2. **Guanciale** (or Pancetta): A type of Italian cured meat, typically made from pork jowl or neck. It's air-dried and has a delicate, unsmoked flavor. Guanciale is the traditional choice, but pancetta is often used as a substitute.
3. **Eggs**: Large, fresh eggs are used to create the creamy sauce.
4. **Parmesan Cheese** (Parmigiano-Reggiano): A hard, aged Italian cheese

💡 KEY CONCEPT: This is a 'stateless' call - the LLM has no memory!


In [4]:
# ================================
# LESSON 2: PROMPT TEMPLATES
# ================================
print("\n" + "="*50)
print("📖 LESSON 2: Prompt Templates - Your First Tool")
print("="*50)

# Why templates? Consistency + Reusability + Dynamic content
cooking_template = """You are a professional chef and cooking instructor.
Your expertise is in {cuisine_type} cuisine.

Question: {user_question}

Please provide a helpful, detailed answer about cooking. Include:
- Key techniques
- Important tips
- Common mistakes to avoid

Answer:"""

# Create the template object
prompt = PromptTemplate(
    input_variables=["cuisine_type", "user_question"],
    template=cooking_template
)

print("📝 Template created! Let's see it in action...")

# Format the template with actual values
formatted_prompt = prompt.format(
    cuisine_type="Italian",
    user_question="How do I make perfect risotto?"
)

print("🎯 Formatted Prompt:")
print(formatted_prompt)

# Use the formatted prompt with LLM
response = llm.invoke(formatted_prompt)
print("\n🤖 LLM Response:")
print(response.content)

print("\n💡 KEY CONCEPT: Templates make prompts reusable and consistent!")


📖 LESSON 2: Prompt Templates - Your First Tool
📝 Template created! Let's see it in action...
🎯 Formatted Prompt:
You are a professional chef and cooking instructor.
Your expertise is in Italian cuisine.

Question: How do I make perfect risotto?

Please provide a helpful, detailed answer about cooking. Include:
- Key techniques
- Important tips
- Common mistakes to avoid

Answer:

🤖 LLM Response:
Risotto, the crown jewel of Italian cuisine! Making a perfect risotto requires attention to detail, patience, and a few key techniques. As a professional chef and cooking instructor, I'm happy to share my expertise with you.

**Key Techniques:**

1. **Heat Control:** Risotto is all about heat control. You want to maintain a consistent, gentle heat throughout the cooking process. This is crucial for developing the creamy texture and preventing the rice from burning or becoming mushy.
2. **Stirring:** Stirring is essential for distributing heat evenly and preventing the rice from sticking to the

In [5]:
# ================================
# LESSON 3: OUTPUT PARSING
# ================================
print("\n" + "="*50)
print("📖 LESSON 3: Output Parsing - Structure Your Responses")
print("="*50)

# Custom output parser for structured responses
class CookingOutputParser(BaseOutputParser):
    """Parse cooking advice into structured format"""

    def parse(self, text: str) -> dict:
        """Extract structured info from LLM response"""
        lines = text.strip().split('\n')

        result = {
            "main_answer": "",
            "techniques": [],
            "tips": [],
            "warnings": []
        }

        current_section = "main_answer"

        for line in lines:
            line = line.strip()
            if not line:
                continue

            # Detect sections
            if "technique" in line.lower():
                current_section = "techniques"
            elif "tip" in line.lower():
                current_section = "tips"
            elif "mistake" in line.lower() or "avoid" in line.lower():
                current_section = "warnings"
            else:
                # Add content to current section
                if current_section == "main_answer" and not result["main_answer"]:
                    result["main_answer"] = line
                elif current_section != "main_answer":
                    result[current_section].append(line)

        return result

    @property
    def _type(self) -> str:
        return "cooking_parser"

# Test the parser
parser = CookingOutputParser()

# Enhanced prompt for structured output
structured_template = """You are a professional chef. Answer the cooking question with this structure:

Main Answer: [Your main response]

Key Techniques:
- [Technique 1]
- [Technique 2]

Pro Tips:
- [Tip 1]
- [Tip 2]

Common Mistakes to Avoid:
- [Mistake 1]
- [Mistake 2]

Question: {question}"""

structured_prompt = PromptTemplate(
    input_variables=["question"],
    template=structured_template
)

# Chain: Prompt → LLM → Parser
question = "How do I make perfect pasta?"
formatted = structured_prompt.format(question=question)
response = llm.invoke(formatted)
parsed_result = parser.parse(response.content)

print("📊 Structured Output:")
for key, value in parsed_result.items():
    print(f"{key.title()}: {value}")

print("\n💡 KEY CONCEPT: Parsers convert raw text into structured data!")


📖 LESSON 3: Output Parsing - Structure Your Responses
📊 Structured Output:
Main_Answer: Main Answer: Making perfect pasta is all about achieving the right balance of texture and flavor. Start by choosing high-quality pasta made from durum wheat semolina, as it will hold its shape better and have a more satisfying bite. Then, cook the pasta al dente, which means it should still have a bit of bite or chew to it. This is important because overcooking can make the pasta mushy and unappetizing.
Techniques: ['- Use a large pot and plenty of salted water to cook the pasta. This will help to season the pasta evenly and prevent it from sticking together.', "- Stir the pasta occasionally while it's cooking to prevent it from settling at the bottom of the pot and sticking together."]
Tips: []
Warnings: []

💡 KEY CONCEPT: Parsers convert raw text into structured data!


In [6]:
# ================================
# LESSON 4: MEMORY SYSTEMS
# ================================
print("\n" + "="*50)
print("📖 LESSON 4: Memory - Making Your Bot Remember")
print("="*50)

# Initialize conversation memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

print("🧠 Memory initialized!")

# Conversation template with memory
conversation_template = """You are a helpful cooking assistant specializing in {cuisine_type} cuisine.
You remember our previous conversation and can reference it.

Previous conversation:
{chat_history}

Current question: {input}

Helpful answer:"""

# Create conversation chain
conversation_prompt = PromptTemplate(
    input_variables=["chat_history", "input", "cuisine_type"],
    template=conversation_template
)

# Manual conversation chain (to understand the mechanics)
def chat_with_memory(user_input, cuisine="Italian"):
    """Simulate conversation with memory"""

    # Get chat history
    chat_history = memory.chat_memory.messages
    history_text = ""

    for message in chat_history:
        if isinstance(message, HumanMessage):
            history_text += f"Human: {message.content}\n"
        elif isinstance(message, AIMessage):
            history_text += f"Assistant: {message.content}\n"

    # Format prompt with memory
    formatted_prompt = conversation_prompt.format(
        chat_history=history_text,
        input=user_input,
        cuisine_type=cuisine
    )

    # Get response
    response = llm.invoke(formatted_prompt)

    # Save to memory
    memory.chat_memory.add_user_message(user_input)
    memory.chat_memory.add_ai_message(response.content)

    return response.content

# Test conversation with memory
print("🗣️ Starting conversation...")

response1 = chat_with_memory("What spices are essential for Italian cooking?")
print("Human: What spices are essential for Italian cooking?")
print(f"Bot: {response1}\n")

response2 = chat_with_memory("Which of those spices work best with seafood?")
print("Human: Which of those spices work best with seafood?")
print(f"Bot: {response2}\n")

response3 = chat_with_memory("Can you suggest a simple recipe using those spices?")
print("Human: Can you suggest a simple recipe using those spices?")
print(f"Bot: {response3}")

print("\n💡 KEY CONCEPT: Memory allows context-aware conversations!")


📖 LESSON 4: Memory - Making Your Bot Remember
🧠 Memory initialized!
🗣️ Starting conversation...


/tmp/ipython-input-6-743683196.py:9: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


Human: What spices are essential for Italian cooking?
Bot: Buon giorno! I'm happy to help you with that. We've talked before, remember? You were curious about Italian cooking, and I shared some tips with you.

Now, when it comes to essential spices for Italian cooking, there are a few that stand out in my mind. You can't go wrong with these basics:

1. Oregano: Ah, the queen of Italian herbs! Oregano adds depth and warmth to many dishes, from pasta sauces to pizza crusts.
2. Basil: Fresh or dried, basil is a staple in Italian cooking. It's the perfect addition to caprese salads, pesto sauces, and more.
3. Thyme: A classic Italian herb, thyme is often used in combination

Human: Which of those spices work best with seafood?
Bot: Seafood in Italian cuisine, how delightful! Given the options we discussed earlier, I think oregano and thyme would be excellent choices to pair with seafood.

Oregano, in particular, pairs beautifully with seafood, especially when combined with garlic and lemon

In [8]:
# ================================
# LESSON 5: PUTTING IT ALL TOGETHER
# ================================
print("\n" + "="*50)
print("📖 LESSON 5: Complete Q&A Bot Class")
print("="*50)

class CookingQABot:
    """Complete Q&A Bot with all components"""

    def __init__(self, cuisine_type="Italian"):
        self.cuisine_type = cuisine_type
        self.llm = ChatGroq(model="llama3-8b-8192", temperature=0.7)
        self.memory = ConversationBufferMemory(return_messages=True)
        self.parser = CookingOutputParser()

        # Main conversation prompt
        self.prompt = PromptTemplate(
            input_variables=["chat_history", "question", "cuisine_type"],
            template="""You are an expert {cuisine_type} cooking instructor.

Previous conversation:
{chat_history}

Student question: {question}

Provide helpful cooking advice. Be specific and practical.

Answer:"""
        )

    def ask(self, question: str) -> str:
        """Ask the bot a question"""

        # Get formatted chat history
        history = self._get_chat_history()

        # Format prompt
        formatted_prompt = self.prompt.format(
            chat_history=history,
            question=question,
            cuisine_type=self.cuisine_type
        )

        # Get response
        response = self.llm.invoke(formatted_prompt)

        # Save to memory
        self.memory.chat_memory.add_user_message(question)
        self.memory.chat_memory.add_ai_message(response.content)

        return response.content

    def _get_chat_history(self) -> str:
        """Format chat history for prompt"""
        messages = self.memory.chat_memory.messages
        history = ""

        for msg in messages[-6:]:  # Last 6 messages to avoid token limits
            if isinstance(msg, HumanMessage):
                history += f"Student: {msg.content}\n"
            elif isinstance(msg, AIMessage):
                history += f"Instructor: {msg.content}\n"

        return history

    def reset_memory(self):
        """Clear conversation history"""
        self.memory.clear()

    def change_cuisine(self, new_cuisine: str):
        """Change the cuisine focus"""
        self.cuisine_type = new_cuisine


📖 LESSON 5: Complete Q&A Bot Class


In [9]:

# ================================
# FINAL DEMO: Interactive Bot
# ================================
print("🚀 Creating your complete Q&A bot...")

# Initialize bot
bot = CookingQABot("Italian")

print("✅ Bot ready! Let's have a conversation...")

# Demo conversation
questions = [
    "What's the secret to good pizza dough?",
    "How long should I knead it?",
    "What if my dough is too sticky?",
    "Can you suggest toppings for a beginner?"
]

for i, question in enumerate(questions, 1):
    print(f"\n--- Question {i} ---")
    print(f"You: {question}")
    response = bot.ask(question)
    print(f"Bot: {response}")


🚀 Creating your complete Q&A bot...
✅ Bot ready! Let's have a conversation...

--- Question 1 ---
You: What's the secret to good pizza dough?
Bot: Bellissima! The secret to good pizza dough is all about the ingredients, the technique, and the patience. Let me share some secrets with you!

First, you need to start with high-quality ingredients. Use "00" flour, which is finely milled and has a lower protein content, making it perfect for pizza dough. Also, use a combination of warm water and yeast to help the dough rise. And don't forget to add a pinch of salt to balance the flavors!

Now, let's talk about technique. The key is to mix the ingredients just until they come together in a shaggy dough. Don't overmix! Stop mixing as soon as the dough starts to form a ball. Then, let it rest for at least 24 hours in the refrigerator to allow the gluten to relax. This step is crucial to creating a tender, easy-to-stretch crust.

Here's a simple recipe to get you started:

Ingredients:

* 1 cup 

In [11]:

# ================================
# LEARNING SUMMARY & NEXT STEPS
# ================================
print("\n" + "="*60)
print("🎓 CONGRATULATIONS! You've learned:")
print("="*60)
print("""
✅ LLM Integration (ChatGroq)
✅ Prompt Templates (structured, reusable prompts)
✅ Output Parsing (structured responses)
✅ Memory Systems (conversation context)
✅ Chain Architecture (connecting components)

🎯 KEY LANGCHAIN CONCEPTS MASTERED:
- Templates separate logic from content
- Memory enables context-aware conversations
- Parsers structure unstructured LLM output
- Chains connect components in pipelines

🚀 NEXT STEPS FOR LOCAL PYTHON:
1. Create a Streamlit interface
2. Add vector database for knowledge retrieval
3. Implement different memory types
4. Add error handling and validation
5. Deploy as a web app

💡 PROFESSOR'S TIP:
The concepts you learned here scale to complex AI applications.
Master these basics, and you can build anything!
""")

# Test your understanding
print("\n🧪 TEST YOUR UNDERSTANDING:")
print("Try modifying the bot to:")
print("1. Focus on a different cuisine")
print("2. Change the conversation style")
print("3. Add new output parsing rules")
print("4. Experiment with different memory limits")

# Example: Change cuisine
bot.change_cuisine("Mexican")
final_response = bot.ask("What are the key spices in Mexican cooking?")
print(f"\n🌮 Mexican Cooking Bot: {final_response}")


🎓 CONGRATULATIONS! You've learned:

✅ LLM Integration (ChatGroq)
✅ Prompt Templates (structured, reusable prompts)  
✅ Output Parsing (structured responses)
✅ Memory Systems (conversation context)
✅ Chain Architecture (connecting components)

🎯 KEY LANGCHAIN CONCEPTS MASTERED:
- Templates separate logic from content
- Memory enables context-aware conversations  
- Parsers structure unstructured LLM output
- Chains connect components in pipelines

🚀 NEXT STEPS FOR LOCAL PYTHON:
1. Create a Streamlit interface
2. Add vector database for knowledge retrieval
3. Implement different memory types
4. Add error handling and validation
5. Deploy as a web app

💡 PROFESSOR'S TIP: 
The concepts you learned here scale to complex AI applications. 
Master these basics, and you can build anything!


🧪 TEST YOUR UNDERSTANDING:
Try modifying the bot to:
1. Focus on a different cuisine
2. Change the conversation style
3. Add new output parsing rules
4. Experiment with different memory limits

🌮 Mexican C